In [89]:
from bs4 import BeautifulSoup
from scipy.spatial import cKDTree as KDTree
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from pyvirtualdisplay import Display

import csv
import cv2
import datetime
import matplotlib
import multiprocessing as mp
import numpy as np
import re
import string
import sys
import unidecode
import traceback
import time

In [90]:
gridCount=1
rois=[]
PATH='/home/abhiavk/git/Website-Evolution/'

In [91]:
def timeTaken(startTime, Metric, MetricValue=""):
	print(Metric.ljust(25," "),datetime.datetime.now()-startTime,"\t\t",MetricValue)

In [92]:
def string_to_words(txt):
    txt=re.sub("[^\w]"," ",  txt)
    txt=re.sub("[0-9]+"," ",  txt)
    txt=re.sub(" [a-zA-Z]{1} "," ",  txt)
    txt=re.sub(" [a-zA-Z]{2} "," ",  txt)
    txt=re.sub("[ ]+"," ",txt)
    return txt.split()

In [93]:
def get_words(d):
    txt=""
    try:
        txt+=d.execute_script("return document.body.innerText;")
    except:
        pass
    try:
        txt+=d.execute_script("return document.innerText;")
    except:
        pass
    words = string_to_words(str(unidecode.unidecode(txt)))
    return words

In [94]:
def get_word_count(d):
    startTime=datetime.datetime.now()
    words=get_words(d)
    wordCount=float(len(words))
    #timeTaken(startTime,"Word Count",wordCount)
    return wordCount

In [95]:
def get_text_body_ratio(soup,wordCount):

    startTime=datetime.datetime.now()
    headers=[]
    for i in range(1,7):
        headers+=soup.findAll("h"+str(i))
    sizeHeaders=[]
    sizeHeaders+=soup.findAll("font",{"size":"3"})
    sizeHeaders+=soup.findAll("font",{"size":"4"})
    sizeHeaders+=soup.findAll("font",{"size":"5"})
    txt=""
    for i in headers:
        txt+=" "+i.text
    for i in sizeHeaders:
        txt+=" "+i.text
    words=[]
    if len(txt)!=0:
        words=string_to_words(str(unidecode.unidecode(txt)))
    #print words
    try:
        headTextCount=float(len(words))
    except:
        headtextCount=0.0
    if wordCount:
        textBodyRatio=headTextCount/wordCount
    else:
        textBodyRatio=0.0
    #timeTaken(startTime,"Text Body Ratio",textBodyRatio)
    return textBodyRatio

In [96]:
def get_emph_body_text_percentage(d,bs,wordCount):

    #print "Param3"
    startTime=datetime.datetime.now()
    boldText = bs.findAll("b")
    words=[]
    for i in boldText:
        try:
            words+= string_to_words(str(unidecode.unidecode(i.text)))
        except:
            pass
    boldWordCount=len(words)
    try:
        txt=str(unidecode.unidecode(d.execute_script("return document.body.innerText")))
    except:
        txt=str(unidecode.unidecode(d.execute_script("return document.body.textContent")))
    pattern = re.compile("!+")
    exclWordCount=len(re.findall(pattern,txt))
    words=get_words(d)
    capWordCount=0
    for i in words:
        if i==i.upper():
            capWordCount+=1

    #print boldWordCount, exclWordCount, capWordCount

    emphTextCount=float(boldWordCount + exclWordCount + capWordCount)

    if wordCount:
        emphTextPercent=(emphTextCount/wordCount)*100.0
    else:
        emphTextPercent=0.0
    #timeTaken(startTime,"Emph text Percent",emphTextPercent)
    return emphTextPercent

In [97]:
def get_text_position_changes(s):
    startTime=datetime.datetime.now()
    #print "Param
    elem=s.findAll()
    prev=""
    textPositionChanges=0
    for i in elem:
        try:
            string=str(i["style"])
            if "text-align:"in string:
                align=string.split("text-align:")[1]
                position=align.split(";")[0].strip()
                if position!=prev:
                    textPositionChanges+=1
                    prev=position
        except:
            pass
    #timeTaken(startTime,"Text Positional Changes",textPositionChanges)
    return textPositionChanges

In [98]:
def get_text_clusters(d,bs):

	#print "Param5"
	startTime=datetime.datetime.now()
	tableText= bs.findAll("td")+bs.findAll("table")
	paraText = bs.findAll("p")
	textClusters=len(tableText)+len(paraText)
	#timeTaken(startTime,"Text Clusters",textClusters)
	return textClusters

In [99]:
def get_visible_links(d,bs):

	#print "Param6"
	startTime=datetime.datetime.now()
	links=bs.findAll("a")
	visibleLinkCount=0
	for i in links:
		if i.text != "":
			visibleLinkCount+=1
	#timeTaken(startTime,"Visible Links",visibleLinkCount)
	return visibleLinkCount

In [100]:
def get_page_size(d):

	#print "Param7"
	startTime=datetime.datetime.now()
	scriptToExecute = "	var performance = 	window.performance ||\
											window.mozPerformance ||\
											window.msPerformance ||\
									 		window.webkitPerformance || {};\
						var network 	= 	performance.getEntries() || {};\
						return network;"
	networkData = d.execute_script(scriptToExecute)
	pageSize=0
	for i in networkData:
		try:
			pageSize+=float(i[u'transferSize'])
		except:
			pass
	pageSize=float(pageSize)/1024.0
	#timeTaken(startTime,"Page Size",pageSize)
	return pageSize

In [101]:
def get_graphics_percent(d,pageSize):

	#print "Param8"
	startTime=datetime.datetime.now()
	scriptToExecute = "var performance = window.performance || window.mozPerformance || window.msPerformance || window.webkitPerformance || {}; var network = performance.getEntries() || {}; return network;"
	networkData = d.execute_script(scriptToExecute)
	graphicsSize=0.0
	for i in networkData:
		try:
			if i[u'initiatorType']== u'script' or i[u'initiatorType']==u'img' or i['initiatorType']== u'css':
				graphicsSize+=float(i[u'transferSize'])
		except:
			pass
	graphicsSize=float(graphicsSize)/1024.0

	if pageSize==0:
		graphicsPercent=0.0
	else:
		graphicsPercent=graphicsSize*100.0/pageSize
	#timeTaken(startTime,"Graphic Size",graphicsSize)
	return graphicsPercent

In [102]:
def get_graphics_count(d,bs):
	startTime=datetime.datetime.now()
	#print "Param9"
	styleSteets=bs.findAll("style")
	scripts=bs.findAll("script")
	images=d.execute_script("return document.images;")
	graphicsCount=len(styleSteets)+len(images)+len(scripts)
	#timeTaken(startTime,"Graphics Count",graphicsCount)
	return  graphicsCount

In [103]:
def get_color_count(image):
    startTime=datetime.datetime.now()
    use_colors = matplotlib.colors.cnames
    named_colors = {k: tuple(map(int, (v[1:3], v[3:5], v[5:7]), 3*(16,))) for k, v in use_colors.items()}
    ncol = len(named_colors)
    no_match = named_colors['purple']

    color_tuples = list(named_colors.values())
    color_tuples.append(no_match)
    color_tuples = np.array(color_tuples)

    color_names = list(named_colors)
    color_names.append('no match')

    tree = KDTree(color_tuples[:-1])

    tolerance = np.inf
    dist, idx = tree.query(image, distance_upper_bound=tolerance)

    colCounts = np.bincount(idx.ravel(), None, ncol+1).tolist()
    colNames  = color_names

    colors=[]
    for i in range(len(color_names)):
        colors.append([colCounts[i],color_names[i]])

    colors.sort(reverse=True)

    colorCount=0
    for color in colors:
        if color[0]>=7864: #1% of the pixels
            colorCount+=1
        else:
            break

    #timeTaken(startTime,"Color Count",colorCount)
    return colorCount

In [104]:
def get_font_count(d,bs):
    startTime=datetime.datetime.now()
    divCount=len(bs.findAll("div"))
    diffFont=set([])
    for i in range(divCount):
        fontStr=""
        script='return document.getElementsByTagName("div")['+str(i)+']["style"]'
        fontStr+=d.execute_script(script+'["font"];')+"font"
        fontStr+=d.execute_script(script+'["fontDisplay"];')+"fontDisplay"
        fontStr+=d.execute_script(script+'["fontFamily"];')+"fontFamily"
        fontStr+=d.execute_script(script+'["fontFeatureSettings"];')+"fontFeatureSettings"
        fontStr+=d.execute_script(script+'["fontKerning"];')+"fontKerning"
        fontStr+=d.execute_script(script+'["fontSize"];')+"fontSize"
        fontStr+=d.execute_script(script+'["fontStretch"];')+"fontStretch"
        fontStr+=d.execute_script(script+'["fontStyle"];')+"fontStyle"
        fontStr+=d.execute_script(script+'["fontVariant"];')+"fontVariant"
        fontStr+=d.execute_script(script+'["fontVariantCaps"];')+"fontVariantCaps"
        fontStr+=d.execute_script(script+'["fontVariantEastAsian"];')+"fontVariantEastAsian"
        fontStr+=d.execute_script(script+'["fontVariantLigatures"];')+"fontVariantLigatures"
        fontStr+=d.execute_script(script+'["fontVariantNumeric"];')+"fontVariantNumeric"
        fontStr+=d.execute_script(script+'["fontVariationSettings"];')+"fontVariationSettings"
        fontStr+=d.execute_script(script+'["fontWeight"];')+"fontWeight"

        diffFont.add(fontStr)
    #print(diffFont)
    fontCount=len(diffFont)-1 # -1 for empty font (default font)
    #timeTaken(startTime,"Font Count",fontCount)
    return fontCount

In [105]:
def getColorfullness(image):
    startTime=datetime.datetime.now()
    (B, G, R) = cv2.split(image.astype("float"))
    rg = np.absolute(R - G)
    yb = np.absolute(0.5 * (R + G) - B)
    (rbMean, rbStd) = (np.mean(rg), np.std(rg))
    (ybMean, ybStd) = (np.mean(yb), np.std(yb))
    stdRoot = np.sqrt((rbStd ** 2) + (ybStd ** 2))
    meanRoot = np.sqrt((rbMean ** 2) + (ybMean ** 2))
    colourFullness = stdRoot + (0.3 * meanRoot)
    #timeTaken(startTime,"Colourfullness",colourFullness)
    return colourFullness

In [106]:
def getVisualComplexity(image,num):
    startTime=datetime.datetime.now()
    year=sys.argv[-2]
    def splitImage(inImg):
        h,w = inImg.shape[0], inImg.shape[1]
        off1X=0
        off1Y=0
        off2X=0
        off2Y=0
        if w >= h:  #split X
            off1X=0
            off2X=int(w/2)
            img1 = inImg[0:h, 0:off2X]
            img2 = inImg[0:h, off2X:w]
        else:       #split Y
            off1Y=0
            off2Y=int(h/2)
            img1 = inImg[0:off2Y, 0:w]
            img2 = inImg[off2Y:h, 0:w]
        return off1X,off1Y,img1, off2X,off2Y,img2
    def qt(inImg, minStd, minSize, offX, offY):
        global gridCount
        global rois
        h,w = inImg.shape[0], inImg.shape[1]
        m,s = cv2.meanStdDev(inImg)
        if s>=minStd and max(h,w)>minSize:
            oX1,oY1,im1, oX2,oY2,im2 = splitImage(inImg)
            gridCount+=1
            qt(im1, minStd, minSize, offX+oX1, offY+oY1)
            qt(im2, minStd, minSize, offX+oX2, offY+oY2)
        else:
            rois.append([offX,offY,w,h,m,s])

    global gridCount
    global rois

    gridCount=1
    rois=[]
    offX, offY=0,0
    minDev        = 10.0
    minSz         = 20

    #cv2.imshow('Start Image',image)
    h,w = image.shape[0], image.shape[1]
    m,s = cv2.meanStdDev(image)
    qt(image,minDev,minSz,offX,offY)
    imgOut=image
    for e in rois:
        col=255
        if e[5]<minDev:
            col=0
        cv2.rectangle(imgOut, (e[0],e[1]), (e[0]+e[2],e[1]+e[3]), col, 1)
    cv2.imwrite(PATH+'webScreenshot/'+str(year)+'/screenshot'+str(num)+'_Quad.png',imgOut)
    #cv2.imshow('Quad Image',imgOut)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    visualComplexity=gridCount#((gridCount*1.0)/(1024.0*768.0))**-1
    #timeTaken(startTime,"Visual Complexity",visualComplexity)
    return visualComplexity

In [107]:
def setDriverOptions():
    options 				= Options()
    options.binary_location = "webEvPy/bin/chromium-browser"
    chrome_driver_binary	= "webEvPy/bin/chromedriver"
    #options.add_argument("--headless")
    return	webdriver.Chrome(options=options)

In [134]:
def getMetrics(urlFile):
    num=urlFile['id']
    url=urlFile['urls']
    #print(url,num)
    startTime 		= datetime.datetime.now()
    textFilename	= PATH+"yearMetrics/CorruptUrls"+str(year)+".txt"
    csvFilename		= PATH+"yearMetrics/tempMpUrlMetrics"+str(year)+".csv"
    try:
        driver			= setDriverOptions()
        driver.get(url)
        try:
            style=driver.find_element_by_xpath("//div[@id='wm-ipp-base']")
            style=driver.execute_script("arguments[0].style.display = 'none'; return arguments[0];", style)
        except:
            print("Hiding wb toolbar error")
            pass
        driver.implicitly_wait(10)
        time.sleep(5)
        driver.set_window_size(1024, 768)
        WebDriverWait(driver, timeout=15).until(lambda x: x.find_elements_by_tag_name('body'))
        
        imagePath=PATH+'webScreenshot/'+str(year)+'/screenshot'+str(num)+'.png'
        #print(imagePath)
        driver.save_screenshot(imagePath)
        image = cv2.imread(imagePath)
        imageGrey = cv2.imread(imagePath,0)
        page_source=driver.page_source
        soup=BeautifulSoup(page_source,'html.parser')
        #---------------------------------------------------#
        #--------- Web Metric Calculation ------------------#
        #---------------------------------------------------#
        wordCount				= get_word_count(driver)#Parameter 1
        textBodyRatio			= get_text_body_ratio(soup,wordCount)#Parameter 2
        emphTextPercent			= get_emph_body_text_percentage(driver,soup,wordCount)#Parameter 3
        textPositionalChanges	= get_text_position_changes(soup)#Parameter 4
        textClusters			= get_text_clusters(driver,soup)#Parameter 5
        visibleLinks			= get_visible_links(driver,soup)#Parameter 6
        pageSize				= get_page_size(driver)#Parameter 7
        graphicsPercent			= get_graphics_percent(driver,pageSize)#Parameter 8
        graphicsCount 			= get_graphics_count(driver,soup)#Parameter 9
        colorCount				= get_color_count(image)#Parameter 10
        fontCount				= get_font_count(driver,soup)#Parameter 11
        colourFullness			= getColorfullness(image)#Parameter 12
        visualComplexity		= getVisualComplexity(imageGrey,num)


        tempMetrics=[
                    num,\
                    wordCount,\
                    textBodyRatio,\
                    emphTextPercent,\
                    textPositionalChanges,\
                    textClusters,\
                    visibleLinks,\
                    pageSize,\
                    graphicsPercent,\
                    graphicsCount,\
                    colorCount,\
                    fontCount,\
                    colourFullness,\
                    visualComplexity,\
                    url
            ]
        line=tempMetrics
        csvFile		= open(csvFilename,"a+")
        csvWriter	= csv.writer(csvFile)
        csvWriter.writerow(line)
        csvFile.close()
        driver.close()
    except:
        print(traceback.format_exc())
        try:
            driver.close()
        except:
            print("###Error : Couldn't close driver")
        print("Error scraping the Url")
        f2			= open(textFilename,"a+")
        f2.write(num+","+url+"\n")
        f2.close()
    print((datetime.datetime.now()-startTime).seconds,"\t",datetime.datetime.now().replace(microsecond=0),"\t",year,num,url)

In [133]:
def main(filename,year=""):
    
    fields			= ["slno","p1","p2","p3","p4","p5","p6","p7","p8","p9","p10","p11","p12","p13","url"]
    csvFilename		= "yearMetrics/tempMpUrlMetrics"+str(year)+".csv"
    csvFile			= open(csvFilename,"a+")
    csvWriter		= csv.writer(csvFile)
    csvWriter.writerow(fields)
    csvFile.close()
    
    fields			= ["id","urls"]
    csvFilename		= "yearMetrics/CorruptUrls"+str(year)+".csv"
    csvFile			= open(csvFilename,"a+")
    csvWriter		= csv.writer(csvFile)
    csvWriter.writerow(fields)
    csvFile.close()
    
    csvFile			= open(filename,"r")
    urlFile			= csv.DictReader(csvFile)
    driver			= setDriverOptions()
    """
    manager 		= mp.Manager()
    urls 			= manager.list()
    results 		= manager.list()
    pool 			= mp.Pool(1)
    results 		= pool.map_async(getMetrics, urlFile)
    while not results.ready():
        pass
    """
    for url in urlFile:
        getMetrics(url)
    csvFile.close()

In [136]:
for year in range(2002,2017):
    filename="yearUrlWb/Wb"+str(year)+".csv"
    year=str(year)
    main(filename,year)

16 	 2019-05-12 08:07:14 	 2002 1 https://web.archive.org/web/20020101000000/http://www.tolerance.org
20 	 2019-05-12 08:07:35 	 2002 2 https://web.archive.org/web/20020101000000/http://www.peta.org
19 	 2019-05-12 08:07:54 	 2002 3 https://web.archive.org/web/20020101000000/http://www.alternet.org
15 	 2019-05-12 08:08:09 	 2002 4 https://web.archive.org/web/20020101000000/http://www.corpwatch.org
13 	 2019-05-12 08:08:23 	 2002 5 https://web.archive.org/web/20020101000000/http://www.idealist.org
14 	 2019-05-12 08:08:38 	 2002 6 https://web.archive.org/web/20020101000000/http://www.guggenheim.com
20 	 2019-05-12 08:08:58 	 2002 7 https://web.archive.org/web/20020101000000/http://www.kazaa.com
19 	 2019-05-12 08:09:18 	 2002 8 https://web.archive.org/web/20020101000000/http://www.120seconds.com
11 	 2019-05-12 08:09:29 	 2002 9 https://web.archive.org/web/20020101000000/http://www.bthere.tv
15 	 2019-05-12 08:09:45 	 2002 10 https://web.archive.org/web/20020101000000/http://www.oddcas

20 	 2019-05-12 08:29:56 	 2002 57 https://web.archive.org/web/20020101000000/http://www.diynet.com
24 	 2019-05-12 08:30:20 	 2002 58 https://web.archive.org/web/20020101000000/http://www.greenhome.com
24 	 2019-05-12 08:30:44 	 2002 59 https://web.archive.org/web/20020101000000/http://www.nationalgardening.com
65 	 2019-05-12 08:31:50 	 2002 60 https://web.archive.org/web/20020101000000/http://www.timeoutny.com
11 	 2019-05-12 08:32:01 	 2002 61 https://web.archive.org/web/20020101000000/http://www.donniedarko.com
27 	 2019-05-12 08:32:29 	 2002 62 https://web.archive.org/web/20020101000000/http://www.ifilm.com
21 	 2019-05-12 08:32:51 	 2002 63 https://web.archive.org/web/20020101000000/http://www.metacritic.com
14 	 2019-05-12 08:33:05 	 2002 64 https://web.archive.org/web/20020101000000/http://www.wakinglifemovie.com
15 	 2019-05-12 08:33:21 	 2002 65 https://web.archive.org/web/20020101000000/http://www.withoutabox.com
18 	 2019-05-12 08:33:39 	 2002 66 https://web.archive.org/we

16 	 2019-05-12 08:55:15 	 2002 130 https://web.archive.org/web/20020101000000/http://www.getty.edu/art/exhibitions/devices
18 	 2019-05-12 08:55:34 	 2002 131 https://web.archive.org/web/20020101000000/http://www.planettribes.com/allyourbase
16 	 2019-05-12 08:55:50 	 2002 132 https://web.archive.org/web/20020101000000/http://www.cutoffmyfeet.com
16 	 2019-05-12 08:56:07 	 2002 133 https://web.archive.org/web/20020101000000/http://www.mindcontrolforums.com
14 	 2019-05-12 08:56:21 	 2002 134 https://web.archive.org/web/20020101000000/http://www.spamradio.com
24 	 2019-05-12 08:56:46 	 2002 135 https://web.archive.org/web/20020101000000/http://www.ology.amnh.org
20 	 2019-05-12 08:57:06 	 2002 136 https://web.archive.org/web/20020101000000/http://www.channelone.com
13 	 2019-05-12 08:57:20 	 2002 137 https://web.archive.org/web/20020101000000/http://www.pinholespy.com
13 	 2019-05-12 08:57:34 	 2002 138 https://web.archive.org/web/20020101000000/http://www.sfskids.com
Traceback (most r

18 	 2019-05-12 09:15:49 	 2003 38 https://web.archive.org/web/20030101000000/http://www.atitd.com
Hiding wb toolbar error
25 	 2019-05-12 09:16:14 	 2003 39 https://web.archive.org/web/20030101000000/http://www.toontown.com
17 	 2019-05-12 09:16:31 	 2003 40 https://web.archive.org/web/20030101000000/http://www.indiegamejam.com
15 	 2019-05-12 09:16:47 	 2003 41 https://web.archive.org/web/20030101000000/http://www.nasa.gov
18 	 2019-05-12 09:17:06 	 2003 42 https://web.archive.org/web/20030101000000/http://www.un.org
20 	 2019-05-12 09:17:26 	 2003 43 https://web.archive.org/web/20030101000000/http://www.weather.gov
23 	 2019-05-12 09:17:50 	 2003 44 https://web.archive.org/web/20030101000000/http://www.publius.org
25 	 2019-05-12 09:18:16 	 2003 45 https://web.archive.org/web/20030101000000/http://www.comune.bologna.it
Traceback (most recent call last):
  File "<ipython-input-134-914ecd78e0ec>", line 20, in getMetrics
    WebDriverWait(driver, timeout=15).until(lambda x: x.find_elem

23 	 2019-05-12 09:38:15 	 2003 88 https://web.archive.org/web/20030101000000/http://www.congress.org
15 	 2019-05-12 09:38:31 	 2003 89 https://web.archive.org/web/20030101000000/http://www.ewg.org
Hiding wb toolbar error
Traceback (most recent call last):
  File "<ipython-input-134-914ecd78e0ec>", line 42, in getMetrics
    fontCount				= get_font_count(driver,soup)#Parameter 11
  File "<ipython-input-104-4b51463ec027>", line 13, in get_font_count
    fontStr+=d.execute_script(script+'["fontSize"];')+"fontSize"
  File "/usr/local/lib/python3.6/site-packages/selenium/webdriver/remote/webdriver.py", line 636, in execute_script
    'args': converted_args})['value']
  File "/usr/local/lib/python3.6/site-packages/selenium/webdriver/remote/webdriver.py", line 321, in execute
    self.error_handler.check_response(response)
  File "/usr/local/lib/python3.6/site-packages/selenium/webdriver/remote/errorhandler.py", line 242, in check_response
    raise exception_class(message, screen, stacktra

...

Error scraping the Url
19 	 2019-05-14 15:12:04 	 2016 137 https://web.archive.org/web/20160101000000/http://www.citizenshipworks.org
27 	 2019-05-14 15:12:32 	 2016 138 https://web.archive.org/web/20160101000000/http://maps.nulawlab.org/
50 	 2019-05-14 15:13:23 	 2016 139 https://web.archive.org/web/20160101000000/https://broadly.vice.com/en_us/
55 	 2019-05-14 15:14:18 	 2016 140 https://web.archive.org/web/20160101000000/http://www.wellandgood.com
45 	 2019-05-14 15:15:04 	 2016 141 https://web.archive.org/web/20160101000000/http://www.sonima.com/
39 	 2019-05-14 15:15:44 	 2016 142 https://web.archive.org/web/20160101000000/http://www.thrillist.com
48 	 2019-05-14 15:16:32 	 2016 143 https://web.archive.org/web/20160101000000/http://gearpatrol.com
48 	 2019-05-14 15:17:21 	 2016 144 https://web.archive.org/web/20160101000000/http://www.newyorker.com
58 	 2019-05-14 15:18:20 	 2016 145 https://web.archive.org/web/20160101000000/http://www.smithsonianmag.com
66 	 2019-05-14 15:19:26

Error scraping the Url
36 	 2019-05-14 15:38:57 	 2016 176 https://web.archive.org/web/20160101000000/http://www.tinytimes.com
20 	 2019-05-14 15:39:18 	 2016 177 https://web.archive.org/web/20160101000000/http://silesoleil.com/
44 	 2019-05-14 15:40:03 	 2016 178 https://web.archive.org/web/20160101000000/http://Www.drakedis.co
41 	 2019-05-14 15:40:44 	 2016 179 https://web.archive.org/web/20160101000000/http://www.factcheck.org/
Hiding wb toolbar error
Traceback (most recent call last):
  File "<ipython-input-134-914ecd78e0ec>", line 20, in getMetrics
    WebDriverWait(driver, timeout=15).until(lambda x: x.find_elements_by_tag_name('body'))
  File "/usr/local/lib/python3.6/site-packages/selenium/webdriver/support/wait.py", line 71, in until
    value = method(self._driver)
  File "<ipython-input-134-914ecd78e0ec>", line 20, in <lambda>
    WebDriverWait(driver, timeout=15).until(lambda x: x.find_elements_by_tag_name('body'))
  File "/usr/local/lib/python3.6/site-packages/selenium/we

Hiding wb toolbar error
36 	 2019-05-14 16:00:28 	 2016 208 https://web.archive.org/web/20160101000000/http://Nixon.com
Hiding wb toolbar error
51 	 2019-05-14 16:01:20 	 2016 209 https://web.archive.org/web/20160101000000/http://barneys.com
56 	 2019-05-14 16:02:16 	 2016 210 https://web.archive.org/web/20160101000000/https://www.helmade.com/
43 	 2019-05-14 16:03:00 	 2016 211 https://web.archive.org/web/20160101000000/http://www.tenslife.com/
54 	 2019-05-14 16:03:55 	 2016 212 https://web.archive.org/web/20160101000000/http://ditto.com
57 	 2019-05-14 16:04:52 	 2016 213 https://web.archive.org/web/20160101000000/http://tumblr.com
21 	 2019-05-14 16:05:14 	 2016 214 https://web.archive.org/web/20160101000000/https://rabb.it
Hiding wb toolbar error
41 	 2019-05-14 16:05:56 	 2016 215 https://web.archive.org/web/20160101000000/http://www.wikihow.com
35 	 2019-05-14 16:06:32 	 2016 216 https://web.archive.org/web/20160101000000/http://ihadcancer.com
Hiding wb toolbar error
Traceback (

27 	 2019-05-14 16:47:09 	 2016 265 https://web.archive.org/web/20160101000000/http://www.theguardian.com/us-news/ng-interactive/2015/jun/01/the-counted-police-killings-us-database
Hiding wb toolbar error
39 	 2019-05-14 16:47:49 	 2016 266 https://web.archive.org/web/20160101000000/http://Nixon.com
20 	 2019-05-14 16:48:10 	 2016 267 https://web.archive.org/web/20160101000000/https://www.hbonow.com/sign-in
39 	 2019-05-14 16:48:49 	 2016 268 https://web.archive.org/web/20160101000000/http://falter.madebywild.com/#en
70 	 2019-05-14 16:49:59 	 2016 269 https://web.archive.org/web/20160101000000/http://www.giphy.com
29 	 2019-05-14 16:50:29 	 2016 270 https://web.archive.org/web/20160101000000/http://cancerfilms.org/classroom/
23 	 2019-05-14 16:50:52 	 2016 271 https://web.archive.org/web/20160101000000/http://suissemania.ch/
35 	 2019-05-14 16:51:28 	 2016 272 https://web.archive.org/web/20160101000000/http://because-recollection.com/
75 	 2019-05-14 16:52:43 	 2016 273 https://web.ar

55 	 2019-05-14 19:36:54 	 2016 304 https://web.archive.org/web/20160101000000/http://www.theguardian.com/news/series/the-long-read
126 	 2019-05-14 19:39:00 	 2016 305 https://web.archive.org/web/20160101000000/http://www.bloomberg.com/graphics/2015-paul-ford-what-is-code/
22 	 2019-05-14 19:39:23 	 2016 306 https://web.archive.org/web/20160101000000/https://performingarts.withgoogle.com/en_us
Hiding wb toolbar error
16 	 2019-05-14 19:39:40 	 2016 307 https://web.archive.org/web/20160101000000/http://w3w.co/webbys
76 	 2019-05-14 19:40:56 	 2016 308 https://web.archive.org/web/20160101000000/http://www.mtvbump.com/watch
13 	 2019-05-14 19:41:09 	 2016 309 https://web.archive.org/web/20160101000000/http://awardsfacility.com/google/entry
Hiding wb toolbar error
17 	 2019-05-14 19:41:26 	 2016 310 https://web.archive.org/web/20160101000000/http:// http://typetodesign.com
Hiding wb toolbar error
30 	 2019-05-14 19:41:57 	 2016 311 https://web.archive.org/web/20160101000000/https://www.ne